In [3]:
from Bio import SeqIO
import pandas as pd

In [4]:
def extract_sequences_from_gbk(gbk_file,new_file,ffn_file = None, tags = ["locus_tag","protein_id","note"]):
    # ffn = SeqIO.to_dict(SeqIO.parse(ffn_file,"fasta"))
    with open(new_file,"w+") as fasta_file:
        for record in SeqIO.parse(gbk_file, "genbank"):
            for feature in record.features:
                if feature.type == "CDS":
                    for key in tags:
                        if feature.qualifiers.get(key):
                            fasta_file.write(f">{feature.qualifiers.get(key)[0]}\n")
                            fasta_file.write(F"{feature.qualifiers['translation'][0]}\n")
                    # try:
                    #     fasta_file.write(f">{feature.qualifiers['locus_tag'][0]}\n")
                    # try:
                    #     fasta_file.write(f">{feature.qualifiers['protein_id'][0]}\n")
                    # except:
                    #     fasta_file.write(f">{feature.qualifiers['note'][0]}\n")
                    


                    # fasta_file.write(f"{str(ffn[feature.qualifiers['locus_tag'][0]].seq)}\n")

    

In [5]:
# for r in SeqIO.parse("../bgc_comp/quinolicomicin/BGC0002520.gbk","gb"):
#     for features in r.features:
#         if features.type == "CDS":
#             for key in ["locus_tag","protein_id","note"]:
#                     if features.qualifiers.get(key):
#                         print(features.qualifiers.get(key))

## Extract sequences from BGCc

In [82]:
#Genomic Files
ref_file = "../bgc_comp/quinolicomicin/BGC0002520.gbk"

minion_gbk_antismash = "../BGC/minion/contig_3.region001.gbk"
# minion_ffn_prokka = "../data/minion/PROKKA_11092023/PROKKA_11092023.ffn"

illumina_gbk_antismash = "../BGC/illumina/Scaffold_1.region009.gbk"
# illumina_ffn_prokka = "../data/ilumina/PROKKA_11092023/PROKKA_11092023.ffn"

In [7]:
# extract_sequences_from_gbk(
#     gbk_file = ref_file
#     # ffn_file = minion_ffn_prokka,
#     new_file = "../bgc_comp/cinerubin/BGC0002362.fasta"
# )
# extract_sequences_from_gbk(
#     gbk_file = minion_gbk_antismash,
#     # ffn_file = minion_ffn_prokka,
#     new_file = "../bgc_comp/cinerubin/BRA006_minion.fasta"
# )
# extract_sequences_from_gbk(
#     gbk_file = illumina_gbk_antismash,
#     # ffn_file = illumina_ffn_prokka,
#     new_file = "../bgc_comp/cinerubin/BRA006_illumina.fasta"
# )

## Analysis

In [83]:
def filter_blast(blast_df):
    columns = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart","qend", "sstart", "send", "evalue", "bitscore"]
    df = pd.read_csv(blast_df,sep="\t", names=columns)
    idx_min = df.groupby("qseqid")["evalue"].idxmin()
    return df.loc[idx_min]

In [84]:
columns = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart","qend", "sstart", "send", "evalue", "bitscore"]
print(" ".join(columns))

qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore


In [85]:
minion = filter_blast("../bgc_comp/quinolicomicin/minion_blast.csv")
minion = minion[minion.pident>70]
illumina = filter_blast("../bgc_comp/quinolicomicin/illumina_blast.csv")
illumina = illumina[illumina.pident>70]

In [86]:
minion.sort_values("sseqid").head()
minion.to_csv("../tables/quinolicomicin_minion_vs_ref.csv", index = False)

In [87]:
illumina.sort_values("sseqid").head()
illumina.to_csv("../tables/quinolicomicin_illumina_vs_ref.csv", index = False)

In [88]:
from Bio import SeqIO
from Bio.Graphics import GenomeDiagram
from Bio.Graphics.GenomeDiagram import CrossLink
from reportlab.lib import colors
from Bio.SeqFeature import SeqFeature, FeatureLocation

In [89]:
ref_gbk = SeqIO.read(ref_file, "gb")
minion_gbk = SeqIO.read(minion_gbk_antismash, "gb")
illumina_gbk = SeqIO.read(illumina_gbk_antismash, "gb")
records = {rec.name: rec for rec in [minion_gbk, ref_gbk, illumina_gbk]}

In [90]:
minion_vs_ref =  [(i.pident,i.qseqid,i.sseqid) for i in minion.itertuples()]
ref_vs_illumina = [(i.pident,i.sseqid,i.qseqid,) for i in illumina.itertuples()]

In [91]:
color_map = {
    "regulatory":"green",
    "transport":"cyan",
    "other":"lightgrey",
    "biosynthetic":"darkred",
    "biosynthetic-additional":"lightpink"}

In [92]:
def get_feature(features, id, tags=["locus_tag", "gene","note","protein_id"]):
    """Search list of SeqFeature objects for an identifier under the given tags."""
    for f in features:
        for key in tags:
            # tag may not be present in this feature
            for x in f.qualifiers.get(key, []):
                if x == id:
                    return f
    raise KeyError(id)

In [96]:
name = "../figs/BRA006_70_quinolicomicin"
gd_diagram = GenomeDiagram.Diagram(name)
feature_sets = {}
max_len = 0
for i, record in enumerate([minion_gbk, ref_gbk, illumina_gbk]):
    max_len = max(max_len, len(record))
    # Allocate tracks 5 (top), 3, 1 (bottom) for A, B, C
    # (empty tracks 2 and 4 add useful white space to emphasise the cross links
    # and also serve to make the tracks vertically more compressed)
    gd_track_for_features = gd_diagram.new_track(
        5 - 2 * i,
        name="-",
        greytrack=True,
        height=0.5,
        start=0,
        end=len(record),
    )
    assert record.name not in feature_sets
    feature_sets[record.name] = gd_track_for_features.new_set()

In [97]:
records

{'contig_3': SeqRecord(seq=Seq('CTAGCTTCGCGGAAGCACGGTGATGGCCACGATGACGCTGTTGACGACAGTGGC...TGA'), id='contig_3', name='contig_3', description='contig_3', dbxrefs=[]),
 'BGC0002520': SeqRecord(seq=Seq('GATCCGCAGTGCCTCGCGCTGACCCTCAGGGCGCTGACCCGGCAGACGCTGCCG...TGG'), id='BGC0002520', name='BGC0002520', description='Micromonospora sp. AK-AN57 DNA, biosynthetic gene cluster for quinolidomicin A', dbxrefs=[]),
 'Scaffold_1': SeqRecord(seq=Seq('ATGCGACGCGATGACCGTGCCCCGAAGCCCGACGCCCCCGTGTACGAGGGCGCC...TGA'), id='Scaffold_1', name='Scaffold_1', description='Scaffold_1', dbxrefs=[])}

In [98]:
minion_gbk.name

'contig_3'

In [99]:
for X, Y, X_vs_Y in [
    (minion_gbk.name, ref_gbk.name, minion_vs_ref),
    (ref_gbk.name, illumina_gbk.name, ref_vs_illumina),
]:
    features_X = records[X].features
    features_Y = records[Y].features
    set_X = feature_sets[X]
    set_Y = feature_sets[Y]
    for score, x, y in X_vs_Y:
        color = colors.linearlyInterpolatedColor(
            colors.white, colors.firebrick, 50, 100, score
        )
        border = colors.lightgrey
        f_x = get_feature(features_X, x)
        F_x = set_X.add_feature(
            SeqFeature(FeatureLocation(f_x.location.start, f_x.location.end, strand=0)),
            color=color,
            border=border,
        )
        f_y = get_feature(features_Y, y)
        F_y = set_Y.add_feature(
            SeqFeature(FeatureLocation(f_y.location.start, f_y.location.end, strand=0)),
            color=color,
            border=border,
        )
        gd_diagram.cross_track_links.append(CrossLink(F_x, F_y, color, border))


In [100]:
for record in [minion_gbk,ref_gbk,illumina_gbk]:
    gd_feature_set = feature_sets[record.name]
    for feature in record.features:
        if feature.type == "CDS":
            try:
                c = color_map[feature.qualifiers["gene_kind"][0]]
            except:
                c = color_map["other"]
            # Exclude this feature
            gd_feature_set.add_feature(
                feature,
                sigil="BIGARROW",
                color=c,
                label=False,
                # name=str(i + 1),
                label_position="start",
                label_size=6,
                label_angle=0,
        )

In [101]:
gd_diagram.draw(format="linear", pagesize="A4", fragments=1, start=0, end=max_len)
gd_diagram.write(name + ".svg", "SVG")

---

In [102]:
# import matplotlib.pyplot as plt
# names_mapp = {
#     "Regulatory":"green",
#     "Transport":"cyan",
#     "Other":"lightgrey",
#     "Biosynthetic":"darkred",
#     "Biosynthetic-additional":"lightpink"}

# # Criar uma lista de patches para a legenda
# legend_patches = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=label) for label, color in names_mapp.items()]

# # Criar a legenda
# plt.legend(handles=legend_patches)

# # Mostrar o gráfico vazio apenas para exibir a legenda
# plt.axis('off')
# # plt.show()
# plt.savefig("../figs/legend.svg", format = "svg", dpi = 1200)